<a href="https://colab.research.google.com/github/rushil-r/TwitterLinkPredModel/blob/main/nets312_ml_TWITTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we illustrate the use of Machine Learning (ML) for link suggestions in the Facebook (FB) graph. We will follow a traditional ML approach that involves manually selecting features, creating a model based on logistic regression, training it, and then evaluating its performance. Let's start by loading the Facebook graph:

In [ ]:
# Import necessary libraries
from google.colab import drive
import networkx as nx
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
%autosave 60

# Mount Google Drive: This is used if you decided to work in Colab using files in your Google Drive
drive.mount('/content/drive')

# Path to your file on Google Drive: 1) Download facebook_combined.txt.gz; 2) Decompressed as a .txt; 3) Rename and locate wherever you like...
file_path = '/content/drive/My Drive/Colab Notebooks/twitterFull.txt'

# Create a new graph and name it twtr
twtr = nx.DiGraph()# Create a graph from the edgelist file
twtr = nx.read_edgelist(file_path, create_using=nx.DiGraph(), nodetype=int)


Autosaving every 60 seconds
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


First, we need to create features for each pair of nodes that can be used to predict whether a link should exist between them. Some common features for link prediction in social networks include: # of Common Neighbors, Jaccard Coefficient, or
the product of the degrees of the two nodes.

In [ ]:
# Feature Engineering
def compute_features(pair):
    u, v = pair

    neighborsSetU = set(twtr.neighbors(u))
    neighborsSetV = set(twtr.neighbors(v))
    predecessorsSetU = set(twtr.predecessors(u))
    predecessorsSetV = set(twtr.predecessors(v))

    # ALL COMMENTS DOCUMENTING FEATURES ARE BELOW, ABOVE THE RELEVANT METHOD DEF
    print(u, v)

    # Jaccard Coefficient -- same as covered in class, but for both the outNeighborhood (directed edge from u/v)
    # and the inNeighborhood (directed edge from v/u)
    def jcrdDistOutDeg(u, v):
        if not ((not neighborsSetU) or (not neighborsSetV)):
            # unionSet = neighborsSetU | neighborsSetV
            # unUV = len(unionSet)
            inUV = len(neighborsSetU & neighborsSetV)
            return inUV/max(len(neighborsSetU | neighborsSetV), 1)
        return 0

    def jcrdDistInDeg(u, v):
        if not ((not predecessorsSetU) or (not predecessorsSetV)):
            # unionSet = predecessorsSetU | predecessorsSetV
            # unUV = len(unionSet)
            inUV = len(predecessorsSetU & predecessorsSetV)
            return inUV/max(len(predecessorsSetU | predecessorsSetV), 1)
        return 0

    # Cosine Similarity -- for both indegree and outdegree, calculate cossin sim
    def cosSimOutDeg(u, v):
        return len(neighborsSetU & neighborsSetV) / max(np.sqrt(len(neighborsSetU) * len(neighborsSetV)), 1)

    def cosSimInDeg(u, v):
        return len(predecessorsSetU & predecessorsSetV) / max(np.sqrt(len(predecessorsSetU) * len(predecessorsSetV)), 1)

    #Adamic Adar index of each node equal to
    # $A(x,y)=\sum_{u\in N(x)\cap N(y)}\frac{1}{\log|N(u)|}$ (latex)
    # calced for both outNeighbors and inNeighbors
    def adamicAdarOutDeg(u, v):
        combinedOutNeighbors = (neighborsSetU & neighborsSetV)
        if not combinedOutNeighbors:
            return 0
        else:
            inverseDegSum = 0
            for node in combinedOutNeighbors:
                inverseDegSum += (1.0 / max(np.log10(twtr.in_degree(node)), 1))
                # G.in_degree(node) called here bc node is a successor of both u&v, so if we
                # consider 'node' to have 2 'neighborhoods' (N_{in} and N_{out}), then we want to
                # calculate with respect to the (size of the) neighborhood set which includes u and v
            return inverseDegSum

    def adamicAdarInDeg(u, v):
        combinedInNeighbors = (predecessorsSetU & predecessorsSetV)
        if not combinedInNeighbors:
            return 0
        else:
            inverseDegSum = 0
            for node in combinedInNeighbors:
                inverseDegSum += (1.0 / max(np.log10(twtr.out_degree(node)), 1))
                # G.out_degree(node) called here bc node is a predecessor of both u&v, so if we
                # consider 'node' to have 2 'neighborhoods' (N_{in} and N_{out}), then we want to
                # calculate with respect to the (size of the) neighborhood set which includes u and v
            return inverseDegSum

    commonFollowing = len(neighborsSetU | neighborsSetV)

    commonFollowedBy = len(predecessorsSetU | predecessorsSetV)

    jaccardFollowing = jcrdDistOutDeg(u, v)
    jaccardFollowedBy = jcrdDistInDeg(u, v)

    cosineSimFollowing = cosSimOutDeg(u, v)
    cosineSimFollowedBy = cosSimInDeg(u, v)

    adamicAdarFollowing = adamicAdarInDeg(u, v)
    adamicAdarFollowedBy = adamicAdarOutDeg(u, v)


    n = twtr.number_of_nodes()
    # shortest path length between u and v -- if they already have an edge between,
    # them, remove it before calculating. Default max is (n - 1) because that is max
    # number of distinct other nodes that can be visited in dipath
    if nx.has_path(twtr, u, v):
        if twtr.has_edge(u, v):
            twtr.remove_edge(u,v)
            if nx.has_path(twtr, u, v):
                pathUV = nx.shortest_path_length(twtr, u, v)
            else:
                pathUV = n - 1.0
            twtr.add_edge(u,v)
        else:
            pathUV = nx.shortest_path_length(twtr, u,v)
    else:
        pathUV = n - 1.0
    if nx.has_path(twtr, v, u):
        if twtr.has_edge(v, u):
            twtr.remove_edge(v,u)
            if nx.has_path(twtr, v, u):
                pathVU = nx.shortest_path_length(twtr, v, u)
            else:
                pathVU = n - 1.0
            twtr.add_edge(v,u)
        else:
            pathVU = nx.shortest_path_length(twtr, v,u)
    else:
        pathVU = n - 1.0

    # weighted negatively to remain consistent that higher feat values are better
    minShortestPaths = -1.0 * min(pathUV, pathVU)

    # RATIO of the outdegree PrefAttachment and Indegree prefattachement of u and v
    prefAttachmentRatio = (twtr.out_degree(u) * twtr.out_degree(v)) / max(twtr.in_degree(u) * twtr.in_degree(v), 1)



    features = {
        "commonFollowing": commonFollowing,
        "commonFollowedBy": commonFollowedBy,
        "jaccardFollowing": jaccardFollowing,
        "jaccardFollowedBy": jaccardFollowedBy,
        "cosineSimFollowing": cosineSimFollowing,
        "cosineSimFollowedBy": cosineSimFollowedBy,
        "adamicAdarFollowing": adamicAdarFollowing,
        "adamicAdarFollowedBy": adamicAdarFollowedBy,
        "minShortestPaths": minShortestPaths,
        "prefAttachmentRatio": prefAttachmentRatio
    }
    return features




Second, for supervised learning, we need to construct a dataset with examples of connected pair of nodes (positive examples) and disconnected pair of nodes (negative examples). We can use existing links in your graph as positive examples. For negative examples, we can randomly sample pairs of nodes that are not connected. Because the graph is sparse, there are many more negative examples that positive examples. This issue is called 'Class Imbalance' and it can dramatically hurt the performance of our logistic classifier. To overcome this issue, we subsample the number of negative examples so that we have as many negative examples as positive examples.

In [ ]:
def randomizedNeg(G, balance):
    nodes = list(G)
    negSample = []
    while len(negSample) < balance:
        u, v = random.sample(nodes, 2)
        # Check if they form an edge
        if not G.has_edge(u, v):
            negSample.append((u, v))
    return negSample


# Constructing the Dataset
# negative_samples = random.sample(list(nx.non_edges(twtr)), len(positive_samples))
# negative_samples = [(u, v) for u, v in nx.non_edges(twtr) if not twtr.has_edge(u, v)]
# negative_samples = random.sample(negative_samples, len(positive_samples)) # Balancing classes with randomization

positive_samples = [(u, v) for u, v in twtr.edges()]
negative_samples = randomizedNeg(twtr, len(positive_samples))


# Compute features for all samples
all_samples = positive_samples + negative_samples
print(len(all_samples))
labels = [1] * len(positive_samples) + [0] * len(negative_samples)
# features = [compute_features(pair) for pair in all_samples]


features = []
i = 0
for pair in all_samples:
    print(i)
    features.append(compute_features(pair))
    i+=1


# Convert to DataFrame
df = pd.DataFrame(features)
df['label'] = labels

Let's take a look at the dataframe...

In [ ]:
print(df.head())
print(df.tail())
print(df.describe()) # This line shows the statistics of the dataset
print(len(df) - len(df.dropna()))
(f"Number of samples in each class: {len(positive_samples)}")

   commonFollowing  commonFollowedBy  jaccardFollowing  jaccardFollowedBy  \
0              251              2466          0.055777           0.071371   
1              311               374          0.331190           0.296791   
2              296               352          0.418919           0.321023   
3              344               586          0.188953           0.126280   
4              217               380          0.004608           0.134211   

   cosineSimFollowing  cosineSimFollowedBy  adamicAdarFollowing  \
0            0.134010             0.264382            86.486744   
1            0.497869             0.473736            51.666071   
2            0.590583             0.498907            52.300963   
3            0.318192             0.251753            34.123639   
4            0.034179             0.239937            23.110136   

   adamicAdarFollowedBy  minShortestPaths  prefAttachmentRatio  label  
0              4.960350              -2.0             0.024628

'Number of samples in each class: 1768149'

Third, let's split our dataset into training and test sets. A common split ratio is 50% training and 50% testing (since we are not going to do model selection, we do not need a validation subset). We use the training set to train our model and the test set to evaluate the model's performance.

In [ ]:
# Data Splitting
X_train, X_test, y_train, y_test = train_test_split(df.drop('label', axis=1), df['label'], test_size=0.3, random_state=42)

Since we are dealing with a binary classification problem (link exists or not), we can use a simple Logistic Regression.

In [ ]:
# Model Selection and Training
model = LogisticRegression(max_iter=5000)
print(X_train)
model.fit(X_train, y_train)

         commonFollowing  commonFollowedBy  jaccardFollowing  \
1056619              239                42          0.037657   
2548742               11                 5          0.000000   
2673465               13                14          0.000000   
3500158              105                33          0.000000   
3334118               52                38          0.000000   
...                  ...               ...               ...   
2356330              110               262          0.000000   
3511566               17                26          0.000000   
2229084               18                75          0.000000   
2768307                3                 4          0.000000   
2219110               58               119          0.000000   

         jaccardFollowedBy  cosineSimFollowing  cosineSimFollowedBy  \
1056619                0.0            0.194054                  0.0   
2548742                0.0            0.000000                  0.0   
2673465           

LogisticRegression(max_iter=5000)

Once our model is trained, we can evaluate our model's performance on the test set using metrics such as accuracy, precision, recall, F1 score, or ROC AUC. This will give you an idea of how well your model can predict new links in the network.

In [ ]:
# Evaluation
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))
print("Accuracy:", accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98    530376
           1       0.99      0.97      0.98    530514

    accuracy                           0.98   1060890
   macro avg       0.98      0.98      0.98   1060890
weighted avg       0.98      0.98      0.98   1060890

Accuracy: 0.9835402350856356


ACCURACY: 98.35402350856356%

Precision[0] =  0.97

Precision[1] = 0.99

Recall[0] = 0.99

Recall[1] = 0.97

F1-Score[0] = 0.98

F1-Score[1] = 0.98


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic
%cd /content/drive/MyDrive/Colab\ Notebooks/
!ls
!jupyter nbconvert --to pdf nets312_ml_TWITTER.ipynb


Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
texlive-fonts-recommended is already the newest version (2021.20220204-1).
texlive-plain-generic is already the newest version (2021.20220204-1).
texlive-xetex is already the newest version (2021.20220204-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/content/drive/MyDrive/Colab Notebooks
 724_685875_cf_lgb_01_train_wns_2_HlDVaNp.ipynb
'altTimeCopy ROY CIS545_HW5_Fall23_StudentVersion.ipynb'
'Chess Project CIS 545.ipynb'
 chss.Rproj
'Copy of Chess Project CIS 545 (1).ipynb'
'Copy of Chess Project CIS 545.ipynb'
'Copy of HW2_Fall 2023_Student_Version.ipynb'
'Copy of HW3_Fall_2023_Student_Version.ipynb'
'Copy of Large Language Models.ipynb'
'Copy of nets312_ml_TWITTER.ipynb'
'Copy of Template_for CIS5210_midterm notes.ipynb'
 Data_Processing_and_Modeling.ipynb
'EDA Chess Project CIS 545.ipynb'
 FacebookDegree.ipynb
 fbCombined312.ipynb
 FBImport